In [1]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision import datasets

import medmnist
from medmnist import INFO, Evaluator

from matplotlib import pyplot as plt

from gudhi import CubicalComplex
from gudhi.sklearn.cubical_persistence import CubicalPersistence

from medmnist import PneumoniaMNIST

from torch_cube_perslap_faster import ColumnRearrangeRegular

In [2]:
# D = torch.tensor([[0, 1, -1, 0, 0, 0], [1, -1, 0, 0, 0, 0], [0, 0, 0, 1, 0, -1]])
# D = torch.tensor([[0, 1, -1, 0, 0, 0, 0], 
#                   [1, -1, 0, 0, 0, 0, 0], 
#                   [0, 0, 0, 1, 0, -1, 0]])

D = torch.tensor([[0, 1, -1, 0, 0, 0, 0], 
                  [1, -1, 0, 0, 0, 0, 0], 
                  [0, 0, 0, 1, 0, -1, 0],
                  [0, 0, 0, 0, 1, 0, -1]])

In [3]:
D

tensor([[ 0,  1, -1,  0,  0,  0,  0],
        [ 1, -1,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  1,  0, -1,  0],
        [ 0,  0,  0,  0,  1,  0, -1]])

In [4]:
graph_D = ColumnRearrangeRegular(D)

In [5]:
# Find vertices wrt D
graph_D.find_vertices()

graph_D.vertices

[0, 1, 2, 3, 4, 5, 6]

In [6]:
graph_D.visited

In [7]:
graph_D.vertices_visited()
graph_D.visited

{0: False, 1: False, 2: False, 3: False, 4: False, 5: False, 6: False}

In [8]:
graph_D.visited[5]

False

In [9]:
# Find edges wrt D
graph_D.find_edges()

graph_D.edges

{0: [1, 2], 1: [0, 1], 2: [3, 5], 3: [4, 6]}

In [10]:
# Find connected components wrt D
graph_D.connected_cmpts()

graph_D.CC

[[0, 1, 2], [3, 5], [4, 6]]